In [127]:
!pip install Keras-Preprocessing

In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re,json,nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding,SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import  confusion_matrix
import seaborn
from sklearn.feature_extraction.text import CountVectorizer
import plotly as py
import cufflinks as cf
import tensorflow
from tensorflow.keras import Sequential
from plotly.offline import iplot
py.offline.init_notebook_mode(connected=True)
cf.go_offline()
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences as pad
from sklearn.preprocessing import LabelEncoder
import re
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding,SpatialDropout1D
from keras_preprocessing.sequence import pad_sequences as pad
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
!pip install git+https://github.com/paulgavrikov/visualkeras
import visualkeras
from sklearn.metrics import  confusion_matrix
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

  Cloning https://github.com/paulgavrikov/visualkeras to /tmp/pip-req-build-wjl5s9e4
  Running command git clone --filter=blob:none --quiet https://github.com/paulgavrikov/visualkeras /tmp/pip-req-build-wjl5s9e4
  Resolved https://github.com/paulgavrikov/visualkeras to commit cd169b81be347e2090353ad6fe2bd2e1f4020cf4
  Preparing metadata (setup.py) ... done


In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Directory for Rajesh**

In [130]:
file_path="/content/drive/MyDrive/Research Work/EMNLP/Code/Test Folder/Copy of blp23_sentiment_test_with_label.xlsx"

In [131]:
df=pd.read_excel("/content/drive/MyDrive/Research Work/EMNLP/Code/Test Folder/Copy of blp23_sentiment_test_with_label.xlsx")

In [132]:
df.head()

,id,text,label
0,7135,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি ? বাংলাদেশ...,Negative
1,28949,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,Negative
2,10210,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,Neutral
3,9526,Pranoy Sen তখন পাকিস্তান ও আফগানিস্তান ভারতের ...,Neutral
4,2142,আরো কত মিথ্যাচার করবে,Negative


In [133]:
df.drop("id",axis=1,inplace=True)

In [134]:
df.shape

(6707, 2)

In [135]:
print("Total Comment:",len(df),
      "\nTotal Negative :",len(df[df.label =='Negative']),
      "\nTotal Neutral :",len(df[df.label =='Neutral']),
      "\nTotal Positive :",len(df[df.label=='Positive']))

Total Comment: 6707 
Total Negative : 3338 
Total Neutral : 1277 
Total Positive : 2092


My Data is not Label, Now I want to Label My Data using sampling

# **Remove Duplicates**

In [136]:
df.shape

(6707, 2)

In [137]:
df.drop_duplicates(inplace=True)

In [138]:
df.shape

(6671, 2)

Here Are no duplicate values

# **In this section I want to check null value**

In [139]:
df.isnull().sum()

text     0
label    0
dtype: int64

# **Remove Small Text and Long Text**

In [140]:
df['no_char'] = df['text'].apply(lambda Sentence: len(Sentence))

In [141]:
df['length'] = df['text'].apply(lambda x:len(x.split()))

In [142]:
dataset = df.loc[df.length>3]
dataset = dataset.reset_index(drop = True)
print("After Cleaning:","\nRemoved {} Small conversations".format(len(df)-len(dataset)),
      "\nTotal conversations:",len(dataset))

After Cleaning: 
Removed 290 Small conversations 
Total conversations: 6381


In [143]:
dataset.shape

(6381, 4)

In [144]:
newdf = dataset.loc[dataset['length'] <= 60]
newdf = newdf.reset_index(drop = True)
print("After Cleaning:","\nRemoved {} Big conversations".format(len(dataset)-len(newdf)),
      "\nTotal conversations:",len(newdf))

After Cleaning: 
Removed 145 Big conversations 
Total conversations: 6236


In [145]:
newdf.shape

(6236, 4)

In [146]:
df=newdf.copy()

In [147]:
df.shape

(6236, 4)

In [148]:
df.head()

,text,label,no_char,length
0,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি ? বাংলাদেশ...,Negative,113,20
1,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,Negative,102,12
2,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,Neutral,74,14
3,Pranoy Sen তখন পাকিস্তান ও আফগানিস্তান ভারতের ...,Neutral,57,10
4,আরো কত মিথ্যাচার করবে,Negative,21,4


# **Below I have cleaned my dataset (Puncuation,links, emoji, non-character)**

In [149]:
import pandas as pd
import regex as re

def remove_non_bengali_characters(sentence):
    cleaned_sentence = re.sub(r'\P{Bengali}+', ' ', sentence)
    return cleaned_sentence.strip()

In [150]:
df['Sentences'] = df['text'].apply(remove_non_bengali_characters)

In [151]:
df.head()

,text,label,no_char,length,Sentences
0,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি ? বাংলাদেশ...,Negative,113,20,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...
1,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,Negative,102,12,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...
2,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,Neutral,74,14,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...
3,Pranoy Sen তখন পাকিস্তান ও আফগানিস্তান ভারতের ...,Neutral,57,10,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে
4,আরো কত মিথ্যাচার করবে,Negative,21,4,আরো কত মিথ্যাচার করবে


In [152]:
df.drop(["text","no_char","length"],axis=1,inplace=True)

In [153]:
df.head()

,label,Sentences
0,Negative,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...
1,Negative,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...
2,Neutral,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...
3,Neutral,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে
4,Negative,আরো কত মিথ্যাচার করবে


In [154]:
import re

# Define a regular expression pattern to match Bengali characters and spaces
bengali_pattern = re.compile("[\u0980-\u09FF ]+")

# Function to remove non-Bengali characters from a string
def remove_non_bengali_chars(text):
    return "".join(filter(bengali_pattern.match, text))

# Apply the function to each element in your dataset
df['Bengali_Text'] = df['Sentences'].apply(remove_non_bengali_chars)


In [155]:
# df.drop(["Sentences","length"],axis=1,inplace=True)

In [156]:
df.head()

,label,Sentences,Bengali_Text
0,Negative,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...
1,Negative,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...
2,Neutral,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...
3,Neutral,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে
4,Negative,আরো কত মিথ্যাচার করবে,আরো কত মিথ্যাচার করবে


# **Remove Stopwords**

# **This Directory for mam**

In [157]:
# def process_conversations(Conversation):
#     stp = open('/content/drive/MyDrive/MY RESEARCH/EMNLP/Dataset/Stopwords.txt','r',encoding="utf8").read().split()
#     result = Conversation.split()
#     Conversation = [word.strip() for word in result if word not in stp ]
#     Conversation =" ".join(Conversation)
#     Conversation = re.sub('[^\u0980-\u09FF]',' ',str(Conversation))
#     return Conversation

In [158]:
# df['cleaned'] = df['Bengali_Text'].apply(process_conversations)
# sample_data = [10,400]
# for i in sample_data:
#      print('Original:\n',df.Bengali_Text[i],'\nCleaned:\n',
#            df.cleaned[i],'\n','Detected as:-- ',df.label[i],'\n')

In [159]:
def process_conversations(Conversation):
    stp = open('/content/drive/MyDrive/Research Work/EMNLP/Dataset2/Stopwords.txt','r',encoding="utf8").read().split()
    result = Conversation.split()
    Conversation = [word.strip() for word in result if word not in stp ]
    Conversation =" ".join(Conversation)
    Conversation = re.sub('[^\u0980-\u09FF]',' ',str(Conversation))
    return Conversation

In [160]:
df['cleaned'] = df['Bengali_Text'].apply(process_conversations)
sample_data = [10,400]
for i in sample_data:
     print('Original:\n',df.Bengali_Text[i],'\nCleaned:\n',
           df.cleaned[i],'\n','Detected as:-- ',df.label[i],'\n')

Original:
 আমি বিবিসি কে বলবো আপনারা একটা জরিপ করেন আমাদের প্রধানমন্ত্রী এই কথার উপরে হ্যা আর না ভোটের আয়োজন করেন 
Cleaned:
 বিবিসি বলবো আপনারা একটা জরিপ প্রধানমন্ত্রী কথার হ্যা ভোটের আয়োজন 
 Detected as:--  Negative 

Original:
 আরো আছে ৪ টা রডের ৩ টি নিয়া গেছে আর ১ টি রেখে গেছে এটারও বিচার হওয়া লাগবে 
Cleaned:
 আরো ৪ টা রডের ৩ নিয়া ১ এটারও বিচার হওয়া লাগবে 
 Detected as:--  Negative 



In [161]:
df.head()

,label,Sentences,Bengali_Text,cleaned
0,Negative,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...,মুখস্ত শিক্ষা জাতি বাংলাদেশ শিক্ষার সার্টিফিকে...
1,Negative,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...
2,Neutral,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,ছেলের দুর্ভাগ্য সৌভাগ্য জানিনা জ্বর স্কুল পারেনি
3,Neutral,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে,পাকিস্তান আফগানিস্তান ভারতের হয়ে
4,Negative,আরো কত মিথ্যাচার করবে,আরো কত মিথ্যাচার করবে,আরো মিথ্যাচার


# **Stemming**

In [162]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])

df['cleaned']=df['cleaned'].apply(stemming)
df.head()

,label,Sentences,Bengali_Text,cleaned
0,Negative,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...,মুখস্ত শিক্ষা দিয়ে কি করবে এই জাতি বাংলাদেশ শ...,মুখস্ত শিক্ষা জাতি বাংলাদেশ শিক্ষার সার্টিফিকে...
1,Negative,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...
2,Neutral,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,আমার ছেলের দুর্ভাগ্য না সৌভাগ্য জানিনা জ্বর এর...,ছেলের দুর্ভাগ্য সৌভাগ্য জানিনা জ্বর স্কুল পারেনি
3,Neutral,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে,তখন পাকিস্তান ও আফগানিস্তান ভারতের হয়ে যাবে,পাকিস্তান আফগানিস্তান ভারতের হয়ে
4,Negative,আরো কত মিথ্যাচার করবে,আরো কত মিথ্যাচার করবে,আরো মিথ্যাচার


In [163]:
df.drop("Bengali_Text",axis=1,inplace=True)

In [164]:
df.drop("Sentences",axis=1,inplace=True)

In [165]:
print("Total Comment:",len(df),
      "\nTotal Negative :",len(df[df.label =='Negative']),
      "\nTotal Neutral :",len(df[df.label =='Neutral']),
      "\nTotal Positive :",len(df[df.label=='Positive']))

Total Comment: 6236 
Total Negative : 3138 
Total Neutral : 1123 
Total Positive : 1975


# **My Data is not Labeled That's why I am going to Augment my Neutral Data**

In [166]:
df.head()

,label,cleaned
0,Negative,মুখস্ত শিক্ষা জাতি বাংলাদেশ শিক্ষার সার্টিফিকে...
1,Negative,জর্ডানের সাবেক যুবরাজ প্রিন্স হামজার ভিডিও বার...
2,Neutral,ছেলের দুর্ভাগ্য সৌভাগ্য জানিনা জ্বর স্কুল পারেনি
3,Neutral,পাকিস্তান আফগানিস্তান ভারতের হয়ে
4,Negative,আরো মিথ্যাচার


In [167]:
# !pip install bangla-stemmer

In [168]:
# from bangla_stemmer.stemmer import stemmer
# import pandas as pd
# def augment_neutral_data(text):
#     stmr = stemmer.BanglaStemmer()
#     augmented_text = stmr.stem(text)
#     return augmented_text
# neutral_data = df[df['label'] == 'Neutral']['cleaned']
# augmented_neutral_data = neutral_data.apply(augment_neutral_data)
# augmented_df = pd.DataFrame({'cleaned': augmented_neutral_data, 'label': 'Neutral'})
# # Concatenate the augmented data with the original DataFrame
# augmented_df = pd.concat([df, augmented_df], ignore_index=True)

# print(augmented_df)

In [169]:
print("Total Comment:",len(df),
      "\nTotal Negative :",len(df[df.label =='Negative']),
      "\nTotal Neutral :",len(df[df.label =='Neutral']),
      "\nTotal Positive :",len(df[df.label=='Positive']))

Total Comment: 6236 
Total Negative : 3138 
Total Neutral : 1123 
Total Positive : 1975


In [170]:
def label_encoding(sentiment,bool):

    le = LabelEncoder()
    le.fit(sentiment)
    encoded_labels = le.transform(sentiment)
    labels = np.array(encoded_labels)
    class_names =le.classes_
    if bool == True:
        print("\n\t\t\t===== Label Encoding =====","\nClass Names:-->",le.classes_)
        for i in sample_data:
            print(sentiment[i],' ', encoded_labels[i],'\n')

    return labels


In [171]:
def dataset_split(feature_space,sentiment):

    X_train,X_test,y_train,y_test = train_test_split(feature_space,sentiment,train_size = 0.8,
                                                  test_size = 0.2,random_state =0)
    print("Feature Size :======>",X_train.shape[1])
    print("\nDataset Distribution:\n")
    print("\tSet Name","\t\tSize")
    print("\t========\t\t======")

    print("\tFull\t\t\t",feature_space.shape[0],
        "\n\tTraining\t\t",X_train.shape[0],
        "\n\tTest\t\t\t",X_test.shape[0])

    return X_train,X_test,y_train,y_test

In [172]:
def calc_gram_tfidf(Conversations):
    tfidf = TfidfVectorizer(ngram_range=(1,3),use_idf=True,tokenizer=lambda x: x.split())
    X = tfidf.fit_transform(Conversations)

    return tfidf,X

In [173]:
def ml_models_defination():

    lr_model = LogisticRegression()
    dt_model = DecisionTreeClassifier()
    rf_model = RandomForestClassifier()
    mnb_model = MultinomialNB(alpha=0.12)
    knn_model = KNeighborsClassifier(n_neighbors=3)
    svm_model = SVC(kernel = 'linear', C=1.01, degree=1, gamma='auto')
    sgd_model = SGDClassifier(loss = 'log',penalty='l2', max_iter=5)
    model_names = ['Logistic Regression','Decision Tree','Random Forest','Naive Bayes','KNN','SVM','sgd']

    ml_models = [lr_model,dt_model,rf_model,mnb_model,knn_model,svm_model,sgd_model]

    return ml_models,model_names

In [174]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def model_performance(model, X_train, X_test, y_train, y_test):
    my_dict = {}
    model.fit(X_train, y_train)
    pred_y = model.predict(X_test)

    # Accuracy
    my_dict['Accuracy'] = round(accuracy_score(y_test, pred_y), 4) * 100

    # Precision, Recall, and F1 Score
    average = 'weighted'  # Choose appropriate average strategy for multi-class
    my_dict['Precision'] = round(precision_score(y_test, pred_y, average=average), 4) * 100
    my_dict['Recall'] = round(recall_score(y_test, pred_y, average=average), 4) * 100
    my_dict['F1 Score'] = round(f1_score(y_test, pred_y, average=average), 4) * 100

    return my_dict


In [175]:
def performance_table(performance_dict):

    acc_list = []
    pr_list = []
    re_list = []
    f1_list = []

    for i in performance_dict.keys():
        acc_list.append(performance_dict[i]['Accuracy'])
        pr_list.append(performance_dict[i]['Precision'])
        re_list.append(performance_dict[i]['Recall'])
        f1_list.append(performance_dict[i]['F1 Score'])

    model_names = ['Logistic Regression','Decision Tree','Random Forest','Multi. Naive Bayes','KNN','SVM','SGD']
    performance_df = pd.DataFrame({'Accuracy':acc_list,'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list,'Model Name':model_names })
    return performance_df

In [176]:
y = ["আমি মনে করি আমি আমার টাকা অপচয়"]
def get_top_n_words(x, n):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    words_freq = [(word, sum_words[0,idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]
get_top_n_words(y,3)


[('আম মন', 1), ('মন কর', 1), ('কর আম', 1)]

In [177]:
z = ["আমি মনে করি আমি আমার টাকা অপচয়"]
def get_top_n_words(x, n):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    words_freq = [(word, sum_words[0,idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]
get_top_n_words(z,3)

[('আম মন কর', 1), ('মন কর আম', 1), ('কর আম আম', 1)]

In [178]:
gram_names = ['Unigram','Bigram','Trigram']
ngrams = [(1,1),(1,2),(1,3)]

for i,gram in enumerate(ngrams):
     tfidf,feature = calc_gram_tfidf(df.cleaned)
     labels = label_encoding(df.label,False)
     X_train,X_test,y_train,y_test = dataset_split(feature,labels)
     ml_models,model_names = ml_models_defination()
     accuracy = {f'{model_names[j]}': model_performance(model, X_train, X_test, y_train, y_test) for j, model in enumerate(ml_models)}

     with open(f'ml_performance_{gram_names[i]}.json', 'w') as f:
           json.dump(accuracy,f)

Feature Size :======> 101456

Dataset Distribution:

	Set Name 		Size
	========		======
	Full			 6236 
	Training		 4988 
	Test			 1248
Feature Size :======> 101456

Dataset Distribution:

	Set Name 		Size
	========		======
	Full			 6236 
	Training		 4988 
	Test			 1248
Feature Size :======> 101456

Dataset Distribution:

	Set Name 		Size
	========		======
	Full			 6236 
	Training		 4988 
	Test			 1248


In [183]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned'], df['label'], test_size=0.2, random_state=42)

# Convert the text data into numerical vectors using CountVectorizer
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)
# Train a Naive Bayes classifier on the training data
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vect, y_train)

# Make predictions on the testing data using the Naive Bayes classifier
y_pred = nb_classifier.predict(X_test_vect)
y_true = np.argmax(y_test, axis=-1)

In [185]:
table = performance_table(accuracy)
df3 =pd.DataFrame(table)
table

,Accuracy,Precision,Recall,F1 Score,Model Name
0,56.73,69.68,56.73,46.82,Logistic Regression
1,49.44,51.59,49.44,50.26,Decision Tree
2,54.57,55.11,54.57,53.20,Random Forest
3,52.96,54.69,52.96,53.28,Multi. Naive Bayes
4,18.59,84.97,18.59,6.06,KNN
5,59.46,60.85,59.46,52.10,SVM
6,57.85,65.49,57.85,49.32,SGD
